In [9]:
import pyspark.sql.functions as F
import pyspark as ps
from pyspark import SQLContext  

spark = ps.sql.SparkSession.builder \
    .master('local[2]') \
    .appName('spark-ml') \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [12]:
df_aapl = sqlContext.read.csv('scores_data.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

df_aapl.show(5) 


+-----------------+----+---+---+---+---+----+------+------+------+----+------+---------+-----+-----+----+--------+----+----+----+------+----+
|           PLAYER|TEAM|AGE| GP|  W|  L| MIN|OFFRTG|DEFRTG|NETRTG|AST%|AST/TO|AST RATIO|OREB%|DREB%|REB%|TO RATIO|EFG%| TS%|USG%|  PACE| PIE|
+-----------------+----+---+---+---+---+----+------+------+------+----+------+---------+-----+-----+----+--------+----+----+----+------+----+
|Russell Westbrook| OKC| 28| 81| 46| 35|34.6| 107.9| 104.6|   3.3|54.3|  1.92|     23.4|  5.3| 27.9|16.7|    12.2|47.6|55.4|40.8|102.31|23.0|
| Boban Marjanovic| DET| 28| 35| 16| 19| 8.4| 104.3| 102.4|   1.9| 5.4|   0.9|      5.1| 16.6| 31.3|23.9|     5.7|54.5|60.6|24.8|  97.2|19.6|
|Demetrius Jackson| BOS| 22|  5|  1|  4| 3.4| 124.2| 117.8|   6.3|30.0|   0.0|     31.1|  9.1| 11.8|10.3|     0.0|87.5|75.3|17.2| 87.46|19.4|
|    Anthony Davis| NOP| 24| 75| 31| 44|36.1| 104.2| 102.5|   1.7|11.0|  0.87|      7.3|  6.7| 26.9|17.0|     8.4|51.8|58.0|32.6|100.19|19.2|
|     

In [14]:
df_aapl.select('TEAM').distinct().show(5)

+----+
|TEAM|
+----+
| GSW|
| DET|
| LAL|
| NYK|
| LAC|
+----+
only showing top 5 rows



In [16]:
df = df_aapl[df_aapl['TEAM'] == 'DET']
df.count()

15

In [17]:
def scores(df, feature):
    min_value = df.select(feature).rdd.min()[0]
    max_value = df.select(feature).rdd.max()[0]
    quantiles = df.approxQuantile(feature, [0.25, 0.5, 0.75], 0)
    
    print(feature, min_value)
    print(feature, max_value)
    print(feature, quantiles[1])
    print(feature, quantiles[0])
    print(feature, quantiles[2])

In [18]:
df.show(5)

+----------------+----+---+---+---+---+----+------+------+------+----+------+---------+-----+-----+----+--------+----+----+----+-----+----+
|          PLAYER|TEAM|AGE| GP|  W|  L| MIN|OFFRTG|DEFRTG|NETRTG|AST%|AST/TO|AST RATIO|OREB%|DREB%|REB%|TO RATIO|EFG%| TS%|USG%| PACE| PIE|
+----------------+----+---+---+---+---+----+------+------+------+----+------+---------+-----+-----+----+--------+----+----+----+-----+----+
|Boban Marjanovic| DET| 28| 35| 16| 19| 8.4| 104.3| 102.4|   1.9| 5.4|   0.9|      5.1| 16.6| 31.3|23.9|     5.7|54.5|60.6|24.8| 97.2|19.6|
|  Andre Drummond| DET| 23| 81| 36| 45|29.7| 102.6| 108.9|  -6.3| 6.0|  0.59|      6.9| 14.6| 37.3|25.2|    11.6|53.1|51.8|22.4| 96.5|13.6|
|      Beno Udrih| DET| 34| 39| 18| 21|14.4| 103.3|  97.9|   5.4|36.4|  3.64|     34.6|  1.2| 10.2| 5.8|     9.5|49.5|53.5|20.1|95.46|13.0|
|   Tobias Harris| DET| 24| 82| 37| 45|31.3| 104.3| 105.5|  -1.3| 8.7|  1.48|     10.1|  2.6| 15.5| 8.9|     6.9|53.2|56.8|21.8|97.41|11.3|
|       Ish Smith| D

In [25]:
for i in ['W','L','MIN','OFFRTG','DEFRTG','NETRTG']:
    scores(df, i)

W 7
W 37
W 33.0
W 16.0
W 36.0
L 2
L 45
L 41.0
L 19.0
L 43.0
MIN 3.5
MIN 33.3
MIN 17.8
MIN 9.8
MIN 29.7
OFFRTG 100.4
OFFRTG 108.2
OFFRTG 103.7
OFFRTG 101.8
OFFRTG 104.3
DEFRTG 97.9
DEFRTG 110.5
DEFRTG 104.1
DEFRTG 100.5
DEFRTG 107.7
NETRTG -8.8
NETRTG 5.4
NETRTG -1.3
NETRTG -2.8
NETRTG 1.9


In [30]:
sns.boxplot(x = df['W'], y = df['OFFRTG'])

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [29]:
import seaborn as sns
for i in ['W','L','MIN','OFFRTG','DEFRTG','NETRTG']:
    for j in ['W','L','MIN','OFFRTG','DEFRTG','NETRTG']:
        if(i!=j):
            sns.boxplot(x = df[j], y = df[i])

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.